In [ ]:
! pip install bastionlab polars

In [2]:
!wget -O diabetes_train.csv.gz https://github.com/scikit-learn/scikit-learn/raw/98cf537f5c538fdbc9d27b851cf03ce7611b8a48/sklearn/datasets/data/diabetes_data_raw.csv.gz
!wget -O diabetes_target.csv.gz https://github.com/scikit-learn/scikit-learn/raw/98cf537f5c538fdbc9d27b851cf03ce7611b8a48/sklearn/datasets/data/diabetes_target.csv.gz
!gzip -d diabetes_train.csv.gz
!gzip -d diabetes_target.csv.gz

--2023-01-04 15:03:55--  https://github.com/scikit-learn/scikit-learn/raw/98cf537f5c538fdbc9d27b851cf03ce7611b8a48/sklearn/datasets/data/diabetes_data_raw.csv.gz
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/scikit-learn/scikit-learn/98cf537f5c538fdbc9d27b851cf03ce7611b8a48/sklearn/datasets/data/diabetes_data_raw.csv.gz [following]
--2023-01-04 15:03:55--  https://raw.githubusercontent.com/scikit-learn/scikit-learn/98cf537f5c538fdbc9d27b851cf03ce7611b8a48/sklearn/datasets/data/diabetes_data_raw.csv.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7105 (6.9K) [application/octet-stream]
Saving

In [1]:
from bastionlab import Connection, Identity
import polars as pl

/home/kbamponsem/base/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
connection = Connection("localhost")

In [3]:
# load dataset
train_df = pl.read_csv(
    "diabetes_train.csv",
    sep=" ",
    has_header=False,
    new_columns=["age", "sex", "bmi", "bp", "s1", "s2", "s3", "s4", "s5", "s6"],
)
test_df = pl.read_csv(
    "diabetes_target.csv",
    has_header=False,
    new_columns=["target"],
)

In [4]:
train_rdf = connection.client.polars.send_df(train_df)
test_rdf = connection.client.polars.send_df(test_df)

In [7]:
# Perform linear regression on `diabetes` dataset.
from bastionlab.linfa.trainers import LinearRegression

model = connection.client.linfa.train(train_rdf, test_rdf, LinearRegression(), 0.85)
print(model)
res = connection.client.linfa.predict(
    model, [60, 2, 28.2, 112.0, 185, 113.8, 42.0, 4.0, 4.9836, 93]
)
print(res)

FittedModel(identifier=2e3ae158-c3e0-4a11-8054-17bdb3abc821)
  └── LinearRegression(fit_intercept=True)
shape: (1, 1)
┌────────────┐
│ prediction │
│ ---        │
│ f64        │
╞════════════╡
│ 192.289151 │
└────────────┘
